### 7. KNN and Decision Trees Exercise (09.06. and 10 points)

##### In this exercise we use our toy data set with the "totalAmount" feature and the "cartItem" features.

##### To test the classifiers split again in train and test data and test the accuracy of the classifier on the test data.

### Exercise 1: KNN from Scratch (6 Points)

##### Write your own implementation of a KNN classifier, as explained last week. Use the Euclidean distance metric for that and use k=5.

##### Make sure that you normalised the feature before training the model (and also normalize the test data before testing).

In [1]:
import pandas as pd
import numpy as np
import math
import operator

transactions = pd.read_csv('data/data.csv')
transactions.head()

,Unnamed: 0,returnLabel,totalAmount,c_0,c_1,c_2,c_3,c_4,c_5
0,0,0,240,1,0,3,1,3,0
1,1,0,79,0,0,1,0,0,0
2,2,0,24,0,0,1,1,0,0
3,3,0,504,0,3,0,0,1,2
4,4,0,190,1,0,1,1,2,0


In [2]:
del transactions["Unnamed: 0"]
transactions.head()

,returnLabel,totalAmount,c_0,c_1,c_2,c_3,c_4,c_5
0,0,240,1,0,3,1,3,0
1,0,79,0,0,1,0,0,0
2,0,24,0,0,1,1,0,0
3,0,504,0,3,0,0,1,2
4,0,190,1,0,1,1,2,0


In [3]:
from sklearn import preprocessing

transactions = transactions.assign(norm_Amount=preprocessing.normalize([transactions.totalAmount]).transpose())
transactions = transactions.assign(norm_c_0=preprocessing.normalize([transactions.c_0]).transpose())
transactions = transactions.assign(norm_c_1=preprocessing.normalize([transactions.c_1]).transpose())
transactions = transactions.assign(norm_c_2=preprocessing.normalize([transactions.c_2]).transpose())
transactions = transactions.assign(norm_c_3=preprocessing.normalize([transactions.c_3]).transpose())
transactions = transactions.assign(norm_c_4=preprocessing.normalize([transactions.c_4]).transpose())
transactions = transactions.assign(norm_c_5=preprocessing.normalize([transactions.c_5]).transpose())

transactions.head()

,returnLabel,totalAmount,c_0,c_1,c_2,c_3,c_4,c_5,norm_Amount,norm_c_0,norm_c_1,norm_c_2,norm_c_3,norm_c_4,norm_c_5
0,0,240,1,0,3,1,3,0,0.004322,0.004696,0.000000,0.014315,0.004613,0.013428,0.000000
1,0,79,0,0,1,0,0,0,0.001423,0.000000,0.000000,0.004772,0.000000,0.000000,0.000000
2,0,24,0,0,1,1,0,0,0.000432,0.000000,0.000000,0.004772,0.004613,0.000000,0.000000
3,0,504,0,3,0,0,1,2,0.009077,0.000000,0.014256,0.000000,0.000000,0.004476,0.009555
4,0,190,1,0,1,1,2,0,0.003422,0.004696,0.000000,0.004772,0.004613,0.008952,0.000000


In [4]:
del transactions["totalAmount"]
del transactions["c_0"]
del transactions["c_1"]
del transactions["c_2"]
del transactions["c_3"]
del transactions["c_4"]
del transactions["c_5"]
transactions.head()

,returnLabel,norm_Amount,norm_c_0,norm_c_1,norm_c_2,norm_c_3,norm_c_4,norm_c_5
0,0,0.004322,0.004696,0.000000,0.014315,0.004613,0.013428,0.000000
1,0,0.001423,0.000000,0.000000,0.004772,0.000000,0.000000,0.000000
2,0,0.000432,0.000000,0.000000,0.004772,0.004613,0.000000,0.000000
3,0,0.009077,0.000000,0.014256,0.000000,0.000000,0.004476,0.009555
4,0,0.003422,0.004696,0.000000,0.004772,0.004613,0.008952,0.000000


In [5]:
transactions = transactions[['norm_Amount', 'norm_c_0', 'norm_c_1', 'norm_c_2', 'norm_c_3', 'norm_c_4', 'norm_c_5', 'returnLabel']]

In [6]:
transactions.head()

,norm_Amount,norm_c_0,norm_c_1,norm_c_2,norm_c_3,norm_c_4,norm_c_5,returnLabel
0,0.004322,0.004696,0.000000,0.014315,0.004613,0.013428,0.000000,0
1,0.001423,0.000000,0.000000,0.004772,0.000000,0.000000,0.000000,0
2,0.000432,0.000000,0.000000,0.004772,0.004613,0.000000,0.000000,0
3,0.009077,0.000000,0.014256,0.000000,0.000000,0.004476,0.009555,0
4,0.003422,0.004696,0.000000,0.004772,0.004613,0.008952,0.000000,0


In [7]:
X_train =  transactions.sample(frac=0.8,random_state=200)
X_test = transactions.drop(X_train.index)

In [8]:
len(X_train)

24800

In [9]:
X_test.head()

,norm_Amount,norm_c_0,norm_c_1,norm_c_2,norm_c_3,norm_c_4,norm_c_5,returnLabel
3,0.009077,0.000000,0.014256,0.000000,0.000000,0.004476,0.009555,0
4,0.003422,0.004696,0.000000,0.004772,0.004613,0.008952,0.000000,0
5,0.003890,0.009393,0.000000,0.014315,0.000000,0.004476,0.000000,0
17,0.013454,0.004696,0.000000,0.014315,0.000000,0.022381,0.000000,1
18,0.001207,0.000000,0.000000,0.000000,0.000000,0.000000,0.004777,0


In [10]:
def calc_eu_dist(data_a, data_b):
  dist = 0
  length = 7
  for x in range(length):
    dist += np.square(data_a[x] - data_b[x])
  return np.sqrt(dist)

In [11]:
def get_knn(train_set, test_data, k):
  dists = []
  length = len(test_data)-1
  for x in range(len(train_set)):
    
    dist = calc_eu_dist(test_data, train_set[x])
    dists.append((train_set[x], dist))
  dists.sort(key=operator.itemgetter(1))
  knn_list = []
  for x in range(k):
    knn_list.append(dists[x][0])
  return knn_list

In [12]:
def get_sorted_votes(n_list):
  class_votes = {}
  for x in range(len(n_list)):
    response = n_list[x][-1]
    if response in class_votes:
      class_votes[response] += 1
    else:
      class_votes[response] = 1
  
  sorted_votes = sorted(class_votes.items(), key=operator.itemgetter(1), reverse=True)
  return sorted_votes[0][0]

In [13]:
def get_accuracy(test_set, knn_predictions):
  correct_predictions = 0
  for x in range(len(test_set)):
    if test_set[x][-1] == knn_predictions[x]:
      correct_predictions += 1
  return (correct_predictions/float(len(test_set))) * 100

In [14]:
def knn_main():
  train_set = []
  train_set = X_train.values.tolist()
  test_set = []
  test_set = X_test.values.tolist()
  knn_predictions = []
  k = 5
  for x in range(len(test_set)):
    n_list = get_knn(train_set, test_set[x], k)
    result = get_sorted_votes(n_list)
    knn_predictions.append(result)
  knn_accuracy = get_accuracy(test_set, knn_predictions)
  print("Accuracy:",knn_accuracy,"%")

In [15]:
knn_main()

Accuracy: 96.96774193548387 %
